In [ ]:
import tensorflow as tf

import numpy as np
import os
import time


import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## DATA PROVISION

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [ ]:
# # Read, then decode for py2 compat.
# text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# # length of text is the number of characters in it
# print(f'Length of text: {len(text)} characters')

In [ ]:
# # Take a look at the first 250 characters in text
# print(text[:250])

In [ ]:
ngrams=4
text=tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))
text=text.filter(lambda x:len(tf.strings.split(x," "))>ngrams-1)
# text=text.map(lambda x:tf.strings.unicode_decode(x, 'UTF-8'))

In [ ]:
# list(map(lambda x: x.decode("utf-8"),list(text.as_numpy_iterator())))

In [ ]:
vocab_size=16384
tokenizer=tf.keras.preprocessing.text.Tokenizer(
    num_words=vocab_size,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=' ',
    char_level=False,
    oov_token=None,
    analyzer=None,
)

In [ ]:
texts=list(map(lambda x: x.decode("utf-8"),list(text.as_numpy_iterator())))

In [ ]:
tokenizer.fit_on_texts(texts)

In [ ]:
sequences=tokenizer.texts_to_sequences(texts)

In [ ]:
new_sequences=[]
for seq in sequences:
  l=len(seq)
  for i in range(2,ngrams):
    new_sequences.append(seq[:i])
  for i in range(ngrams,l):
    new_sequences.append(seq[i-ngrams:i])

new_sequences = np.array(pad_sequences(new_sequences, maxlen=4, padding='pre'))
new_sequences_x= new_sequences[:,:-1]
new_sequences_y= new_sequences[:,-1]
l=len(new_sequences)
thr=int(l*0.7)

In [ ]:
tokenizer.get_config()["word_index"]

In [ ]:
data_train=tf.data.Dataset.from_tensor_slices((new_sequences_x[:thr],new_sequences_y[:thr]))
data_test=tf.data.Dataset.from_tensor_slices((new_sequences_x[thr:],new_sequences_y[thr:]))

In [ ]:
inp=tf.keras.layers.Input((ngrams-1,))
embedding=tf.keras.layers.Embedding(vocab_size - 1, 64)(inp)
# att1=tf.keras.layers.MultiHeadAttention(num_heads=3,key_dim=32,output_shape=64)(embedding,embedding)
# att2=tf.keras.layers.MultiHeadAttention(num_heads=3,key_dim=32,output_shape=64)(att1,att1)
lstm1=tf.keras.layers.LSTM(128,return_sequences=True)(embedding)
lstm2=tf.keras.layers.LSTM(128)(lstm1)
flatten=tf.keras.layers.Flatten()(lstm2)
out=tf.keras.layers.Dense(vocab_size-1, activation="softmax")(flatten)
model=tf.keras.Model(inputs=inp,outputs=out)
model.summary()

In [ ]:
model.compile(optimizer=tf.optimizers.RMSprop(),loss=tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=False,
        ),metrics=["accuracy"])

checkpoint_filepath = 'temp.ckpt'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


In [ ]:
model.fit(data_train.batch(64),epochs=128,validation_data=data_test.batch(64),callbacks=[model_checkpoint_callback])

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
def predict_next_word(input_sequence):
  seq=tokenizer.texts_to_sequences([input_sequence])[0]
  if len(seq)>4:
    seq=seq[-4:]
  seq=pad_sequences([seq], maxlen=ngrams-1, padding='pre')[0]
  seq=list(seq)
  seq.append(model.predict(np.array([seq]))[0].argmax())
  if len(seq)>4:
    seq=seq[-4:]
  seq_inv=tokenizer.sequences_to_texts([seq])[0]
  input_sequence+=(" "+seq_inv.split(" ")[-1])
  return input_sequence

In [ ]:
seq="god of old"
for i in range(40):
  seq=predict_next_word(seq)
print(seq)

In [ ]:
seq="king of "
for i in range(40):
  seq=predict_next_word(seq)
print(seq)

In [ ]:
# cnt=0
# for el in data:
#   print(el)
#   cnt+=1
#   if cnt==10:
#     break